In [165]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import sys
import copy

In [166]:
# 1  import data 
# convert string to float
X_and_Y_train = np.loadtxt('data/pa2train.txt', delimiter=' ', usecols=range(23))
print "X_and_Y_train", X_and_Y_train.shape

X_and_Y_val = np.loadtxt('data/pa2validation.txt', delimiter=' ', usecols=range(23))
print "X_and_Y_val", X_and_Y_val.shape

X_and_Y_test = np.loadtxt('data/pa2test.txt', delimiter=' ', usecols=range(23))
print "X_and_Y_test is", X_and_Y_test.shape

X_and_Y_train (2000, 23)
X_and_Y_val (1000, 23)
X_and_Y_test is (1000, 23)


In [167]:
# passed in subset has 23 col, 22 features and  1 label
def calc_Entropy(feature,threshold,subset):

    counter1 = 0.0
    counter2 = 0.0
    counter3 = 0.0
    counter4 = 0.0
    trueNum = 0.0
    falseNum = 0.0
    totalNum = float(subset.shape[0])
    
    for i in range(subset.shape[0]):
        if(subset[i,feature] < threshold):
            trueNum+=1
            if (subset[i,22]==1):
                counter1+=1
            else:
                counter2+=1
        else:
            falseNum+=1
            if (subset[i,22]==1):
                counter3+=1
            else:
                counter4+=1
                
    prob1 = counter1/trueNum
    prob2 = counter2/trueNum
    prob3 = counter3/falseNum
    prob4 = counter4/falseNum
    probTrue = trueNum/totalNum
    probFalse = falseNum/totalNum
    
    if ((prob1==0 or prob2==0) and (prob3==0 or prob4==0)):
        entropy = 0
    elif ((prob1==0 or prob2==0) and prob3!=0 and prob4!=0):
        entropy = probFalse*(-prob3*np.log(prob3)-prob4*np.log(prob4))
    elif ((prob3==0 or prob4==0) and prob1!=0 and prob2!=0):
        entropy = probTrue*(-prob1*np.log(prob1)-prob2*np.log(prob2))
    else:
        entropy = probTrue*(-prob1*np.log(prob1)-prob2*np.log(prob2)) + \
        probFalse*(-prob3*np.log(prob3)-prob4*np.log(prob4))
    return entropy

In [168]:
# 2) Implement the DT
# data structure 
class DT_Node:
    def __init__(self,subset):
        self.left = None
        self.right =  None
        self.subset = subset
        self.threshold = None
        self.feature = None
        self.label = None
        self.pure = False

In [169]:
def findIndex(feature, threshold, sorted_by_feature):
    for i in range(sorted_by_feature.shape[0]-1):
        if ((sorted_by_feature[i,feature]) < threshold and (sorted_by_feature[i+1,feature]) > threshold):
            return i

In [170]:
# func: split the subset in a node into two subsets,left child node for the spliting rule 
# to be True and right child for the spliting rule ot be False
# notice that passed in node contains a subset, which is a 2D array, data by (feature+label)
# return: nothing
def splitRule(node):
    
    sorted_subset = copy.deepcopy(node.subset)
    #sort the subset data by all feature, axis=0 sort by col
    sorted_subset.sort(axis=0)
    
    Best_threshold=0
    Best_feature=0
    Min_entropy=sys.maxint

#     print "checking"
    for feature in range(sorted_subset.shape[1]-1):   #last col label not counted
        for data in range(sorted_subset.shape[0]-1):  
            #for all separable threshold, find its entropy
            if(sorted_subset[data,feature] != sorted_subset[data+1,feature]):
                #set threshold
                threshold = (sorted_subset[data,feature]+sorted_subset[data+1,feature])/2
                #calc entropy
                entropy = calc_Entropy(feature,threshold,node.subset)
                #update 
                if(entropy < Min_entropy):
                    Best_feature = feature
                    Best_threshold = threshold
                    Min_entropy = entropy
#                     print "Best_threshold,",Best_threshold, \
#                     "Best_feature",Best_feature,"Min_entropy",Min_entropy
                
    #   sorted by feature 
    sorted_by_feature = node.subset[np.argsort(node.subset[:,Best_feature])] 
    #   find spliting threshold index
    Best_threshold_index = findIndex(Best_feature, Best_threshold, sorted_by_feature)
    #   left child->True,  right child->false  of spliting rule
    left_subset = sorted_by_feature[:(Best_threshold_index+1),:] 
    right_subset = sorted_by_feature[(Best_threshold_index+1):,:]

    # set properties in the node
    node.threshold = Best_threshold
    node.feature = Best_feature
    node.left = DT_Node(left_subset)
    node.right = DT_Node(right_subset)

In [171]:
# func: check if the subset in a node is pure 
# return: Ture/False
def pureNode(node):
    label = node.subset[0,22]
    for i in range (node.subset.shape[0]):
        if(node.subset[i,22]!=label):
            return False
    return True

In [172]:
def ID3_helper(node):

    splitRule(node)
    
    if (pureNode(node.left)==True):  
        node.left.pure = True 
        node.left.label = node.left.subset[0,22]
    else:  
        #recursive call
        ID3_helper(node.left)
        
    if (pureNode(node.right)==True):
        node.right.pure = True 
        node.right.label = node.right.subset[0,22]
    else:  
        #recursive call
        ID3_helper(node.right) 

In [173]:
def ID3_alg(X_and_Y_train):
    #inti   
    root = DT_Node(X_and_Y_train)
    #check if root can split 
    
    #call helper function to split the node recursively
    ID3_helper(root) 
    return root

In [193]:
# 3) call ID3 algorithm to build the tree

root = ID3_alg(X_and_Y_train)


In [194]:
# traverse the tree using bfs
def bfs(root):
    queue = []
    queue.append(root)
    while queue:
        currNode = queue.pop(0)
        if(currNode.pure == False):
            print "--------------------\n Internal node: ",repr(currNode),"\nleft child: ",repr(currNode.left),\
            "\nright child: ",repr(currNode.right),"\nsplit rule: feature",currNode.feature+1," < ",currNode.threshold,\
            "\nsubset size: ",currNode.subset.shape[0]
        else:
            print "--------------------\n Leaf Node: ",repr(currNode),"\n label: ",currNode.label,\
            "\subset size: ",currNode.subset.shape[0]
        if(currNode.left!=None):
            queue.append(currNode.left) 
        if(currNode.right!=None):
            queue.append(currNode.right)

# Part1 graphing the tree

In [195]:
# 4) call bfs to print the tree
bfs(root)

--------------------
 Internal node:  <__main__.DT_Node instance at 0x118c7f0e0> 
left child:  <__main__.DT_Node instance at 0x118c7f320> 
right child:  <__main__.DT_Node instance at 0x118dd4638> 
split rule: feature 5  <  0.5 
subset size:  2000
--------------------
 Internal node:  <__main__.DT_Node instance at 0x118c7f320> 
left child:  <__main__.DT_Node instance at 0x118dd45f0> 
right child:  <__main__.DT_Node instance at 0x118dd4680> 
split rule: feature 1  <  415000.0 
subset size:  1319
--------------------
 Internal node:  <__main__.DT_Node instance at 0x118dd4638> 
left child:  <__main__.DT_Node instance at 0x118d9eab8> 
right child:  <__main__.DT_Node instance at 0x118d9eb00> 
split rule: feature 5  <  1.5 
subset size:  681
--------------------
 Internal node:  <__main__.DT_Node instance at 0x118dd45f0> 
left child:  <__main__.DT_Node instance at 0x118dd46c8> 
right child:  <__main__.DT_Node instance at 0x118dd4710> 
split rule: feature 17  <  2506.5 
subset size:  1284
----

split rule: feature 12  <  3124.0 
subset size:  7
--------------------
 Internal node:  <__main__.DT_Node instance at 0x118c99098> 
left child:  <__main__.DT_Node instance at 0x118c99128> 
right child:  <__main__.DT_Node instance at 0x118c99170> 
split rule: feature 11  <  46792.5 
subset size:  95
--------------------
 Internal node:  <__main__.DT_Node instance at 0x118c990e0> 
left child:  <__main__.DT_Node instance at 0x118c997e8> 
right child:  <__main__.DT_Node instance at 0x118c99830> 
split rule: feature 14  <  5583.0 
subset size:  12
--------------------
 Internal node:  <__main__.DT_Node instance at 0x118c99a28> 
left child:  <__main__.DT_Node instance at 0x118c99ab8> 
right child:  <__main__.DT_Node instance at 0x118c99b00> 
split rule: feature 14  <  25530.0 
subset size:  165
--------------------
 Leaf Node:  <__main__.DT_Node instance at 0x118c99a70> 
 label:  1.0 \subset size:  1
--------------------
 Leaf Node:  <__main__.DT_Node instance at 0x118c99ea8> 
 label:  0.0 

split rule: feature 1  <  150000.0 
subset size:  22
--------------------
 Leaf Node:  <__main__.DT_Node instance at 0x118c7bb00> 
 label:  0.0 \subset size:  9
--------------------
 Leaf Node:  <__main__.DT_Node instance at 0x118c7bf38> 
 label:  1.0 \subset size:  2
--------------------
 Leaf Node:  <__main__.DT_Node instance at 0x118c7bf80> 
 label:  0.0 \subset size:  2
--------------------
 Leaf Node:  <__main__.DT_Node instance at 0x118da41b8> 
 label:  1.0 \subset size:  1
--------------------
 Leaf Node:  <__main__.DT_Node instance at 0x118da4200> 
 label:  0.0 \subset size:  40
--------------------
 Internal node:  <__main__.DT_Node instance at 0x118da4248> 
left child:  <__main__.DT_Node instance at 0x118da42d8> 
right child:  <__main__.DT_Node instance at 0x118da4320> 
split rule: feature 4  <  1.5 
subset size:  52
--------------------
 Leaf Node:  <__main__.DT_Node instance at 0x118da4290> 
 label:  1.0 \subset size:  2
--------------------
 Leaf Node:  <__main__.DT_Node i

# Part2  training and test error

In [196]:
#purpose: func for predicting error given data
#return: error
def predict(data,root):

    wrong_counter = 0.0
    data_size = data.shape[0]
    
    for i in range(data_size):  
        currNode = root
        while(currNode.pure==False):
            if(data[i,currNode.feature] < currNode.threshold):
                currNode=currNode.left
            else:
                currNode=currNode.right
                
        if(currNode.label != data[i,22]):
            wrong_counter+=1

    error = wrong_counter/float(data_size)
    return error

In [197]:
training_error = predict(X_and_Y_train,root)
print "training_error: ",training_error

testing_error = predict(X_and_Y_test,root)
print "testing_error: ",testing_error

val_error = predict(X_and_Y_val,root)
print "validation_error: ",val_error

training_error:  0.0
testing_error:  0.173
validation_error:  0.179


# Part3  validation and test error after 1st and 2nd round pruning

In [201]:
def calcMajorityLabel(data):
    counts = np.bincount([int(i) for i in data[:,22]])
    return np.argmax(counts)

In [202]:
#purpose: for each node v in the tree, replace the tree starting at 
#root v by single node that predicts the majority label in v to get 
#tree T', if error(T')<=error(T), replace T by T'
def pruning(data,root,currNode,prev_error):
  
    #calculate majority
    majority_label = calcMajorityLabel(data)
      
    #replace the node by changing property inside the node
    currNode.label = majority_label
    currNode.pure = True
    
    after_error = predict(data,root)
    
    #change back since no progress made
    if(after_error > prev_error):
        currNode.label = None
        currNode.pure = False
        return False
    else:
        return True

In [203]:
def bfs_purning(data,root):

    prev_error = predict(X_and_Y_val,root)
    print "prev_error:", prev_error
    
    queue = []
    queue.append(root)
    while queue:
        currNode = queue.pop(0)
        flag = pruning(data,root,currNode,prev_error)
#         print "repr(currNode):",repr(currNode),"\nleft:",currNode.left,"\nright:",\
#         currNode.right,"\nsubset:",currNode.subset.shape,"\nlabel:",currNode.label,"\n"
#         #found the pruning node
        if(flag):
            print "--------------------\npruning node: ",repr(currNode), \
            "\nsubset:",currNode.subset.shape[0],"\nlabel:",currNode.label
            break
        if(currNode.left.pure==False):
            queue.append(currNode.left)
        if(currNode.right.pure==False):
            queue.append(currNode.right) 

In [204]:
bfs_purning(X_and_Y_val,root)

validation_error = predict(X_and_Y_val,root)
print "validation_error: ",validation_error

testing_error = predict(X_and_Y_test,root)
print "testing_error: ",testing_error


prev_error: 0.179
--------------------
pruning node:  <__main__.DT_Node instance at 0x118c7f320> 
subset: 1319 
label: 0
validation_error:  0.122
testing_error:  0.117
